# 커스텀모델 함수를 사용해보자

### 인공신경망 만드는 함수
- factiory_model( 트레이닝 데이터(X값=칼럼=피쳐=열) ,  타겟 데이터(맞춰야하는 데이터) , 하이퍼 파라미터 셋팅한 json파일의 이름)
- -> EX :  factory_model(X_trn, y_trn,'config.json')

          리턴값(아웃풋)은 인공신경망 모델을 리턴함 학습도 가능함
          model = factory_model(X_trn, y_trn,'config.json')
          model.fit(X_trn, y_trn)
### 인공신경망 모델 여러개 만드는 함수
- create_models( 트레이닝 데이터(X값=칼럼=피쳐=열) ,  타겟 데이터(맞춰야하는 데이터), 하이퍼 파라미터 셋팅한 json파일의 이름 'configW.json' 무조건 W가 뒤에 붙어있어야함 , 몇개의 config.json파일을 넣을 것인지 )
- -> EX :  create_models(X_trn, y_trn,'configW.json',2)

          리턴값(아웃풋)은 인공신경망 모델 여러개를 담은 리스트를 리턴함 인덱스로 꺼내서 사용 가능함
          models = create_models(X_trn, y_trn,'configW.json',2)
          model1 = models[0]
### 인공신경만 모델에 대한 교차검증 함수
- CV(모델(한개의 모델), 트레이닝 데이터(X값=칼럼=피쳐=열) ,  타겟 데이터(맞춰야하는 데이터), 5번 교차 검증을 할 것이고 기본값이기에 안적어도 됨)
- -> EX :  CV(m, feature, label, n_splits=5)

          리턴값(아웃풋)은 판다스 데이터프레임이다 교차검증 후 5번 평균의 결과값을 가지고 있음 다음과 같이 프레임 확인도 가능함
          d = CV(m, feature, label, n_splits=5)  
          d.head()
### 인공신경만 모델 여러개에 대한 교차검증 함수
- models_cv(모델들(여러개의 모델), 트레이닝 데이터(X값=칼럼=피쳐=열) ,  타겟 데이터(맞춰야하는 데이터), 5번 교차 검증을 할 것이고 기본값이기에 안적어도 됨)
- -> EX :  models_cv(models, feature, label, n_splits=5)

          리턴값(아웃풋)은 판다스 데이터프레임이다 여러 모델들을 순서대로 하나씩 5번 교차검증 후 데이터 평균 결과값을 한 행마다 스택처럼 쌓아서 리턴함
          d = models_cv(models, feature, label, n_splits=5) 
          d.head()

In [6]:
import pandas as pd
import numpy as np
from nn import DynamicANNWrapper, DynamicANN
from sklearn.model_selection import train_test_split
import torch
from factory_model import factory_model, create_models , models_cv
from cv import ClfiCV,RegCV
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE


In [5]:
df = pd.read_csv('data/prepro_jung.csv')

# 타겟값 분리

In [ ]:

# 판다스 데이터프레임을 트레인과 테스트로 나누기
train_df, test_df = train_test_split(df, test_size=0.2, random_state=2023)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=2023)

# 여기서
# - your_dataframe은 나눌 데이터프레임을 대체하는 변수명입니다.
# - test_size는 테스트 셋의 비율을 나타냅니다. 0.2는 80% 트레인, 20% 테스트로 나누겠다는 뜻입니다.
# - random_state는 나누는 과정에서 사용되는 난수 발생 시드(seed)를 의미하며, 동일한 결과를 얻기 위해 설정합니다.

# 결과로 나온 train_df와 test_df를 사용하여 모델을 훈련하고 테스트할 수 있습니다.

In [ ]:
train_df.info()


In [ ]:
X_trn = train_df.drop('타겟',axis=1)
y_trn = train_df['타겟']
X_tst = test_df.drop('타겟',axis=1)
y_tst = test_df['타겟']

In [ ]:
y_trn.value_counts()

In [ ]:
y_tst.value_counts()

In [ ]:
# SMOTE 인스턴스 생성
smote = SMOTE(sampling_strategy='auto', random_state=0)

# 오버샘플링 적용
X_resampled, y_resampled = smote.fit_resample(X_trn, y_trn) #판다스데이터타입도 넘파이로변환하는게 아니라 판다스로 리턴해줌

# 결과 확인
print(pd.Series(y_resampled).value_counts())
X_resampled.shape
print(type(X_resampled), type(y_resampled))


In [ ]:
X_trn = X_trn.values
y_trn = y_trn.values
X_tst = X_tst.values
y_tst = y_tst.values

In [ ]:
X_trn.shape, y_trn.shape

In [ ]:
models = create_models(X_trn, y_trn,'configW.json',2)


m = models[0]
print(m)

In [ ]:
feature = val_df.drop('타겟',axis=1)
label = val_df['타겟']

feature = feature.values
label = label.values


In [ ]:

model = factory_model(feature, label, 'config.json')

In [ ]:
print(model)

In [ ]:
model.fit(X_trn, y_trn)

In [ ]:
predictions = model.predict(X_tst)

In [ ]:
# predicted_classes = np.argmax(predictions, axis=1) 

In [ ]:
accuracy = accuracy_score(y_tst, predictions)

In [ ]:
print(accuracy)

In [ ]:
models = m
d = models_cv(models, feature, label)


In [ ]:
m = create_models(feature, label, "configW.json", 4)


In [ ]:
m = model

In [ ]:
performance_metrics = ClfiCV(m, feature, label, n_splits=5)

RegCV(m, feature, label, n_splits=5)

performance_metrics.head()


In [ ]:
d.head()